<a href="https://colab.research.google.com/github/difli/astra-vsearch-image/blob/main/astra-vsearch-image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# astra-vsearch-image
Jupyter notebook for image search powered by [Astra Vector Search](https://docs.datastax.com/en/astra-serverless/docs/vector-search/overview.html) and OpenAI [CLIP Model](https://github.com/openai/CLIP).

## OpenAI CLIP Model
CLIP, or "Contrastive Language-Image Pretraining", is an artificial intelligence model developed by OpenAI. The model is trained to understand and associate images with natural language by using a vast number of images and their associated textual descriptions. CLIP can perform tasks such as generating textual descriptions of images or finding images based on given text.

## Astra Vector Search
Astra vector search enables developers to search a database by context or meaning rather than keywords or literal values. This is done by using “embeddings”. Embeddings are a type of representation used in machine learning where high-dimensional or complex data is mapped onto vectors in a lower-dimensional space. These vectors capture the semantic properties of the input data, meaning that similar data points have similar embeddings. The CLIP model is used to generate the embeddings in this demo.

## Demo Summary
- The CLIP model generates embeddings for each image 
- Metdata and embeddings for each image are stored in Astra DB. 
- The embeddings are stored in a Astra DB column of Type Vector.  
- The idea of the demo is to find with the help of Astra Vector Search the image of the house with a swimming pool.
- The CLIP model generates embeddings based on the search string: "a house with a swimming pool". 
- This embedding is used in the query to find the image that has similar embeddings as the search string.
- You should see the image of a house with a swimming pool

# Getting Started with this notebook
- Create a new vector search enabled database in Astra.
- Create a keyspace
- Create a token with permissions to create tables
- Download your secure-connect-bundle.zip file.
- Download the [image files](https://github.com/difli/astra-vsearch-image/tree/19b7f1496627b8406b05d3abbea63968d92397f8/images)
- When you open this notebook in Google Colab or your own notebook server, drag-and-drop the secure-connect-bundle.zip and images (one.jpg, two.jpg, three.jpg and for.jpg) into the File Browser of the notebook
- Update the Environment Variables cell in the notebook with information from the token you generated, the name of your secure-connect-bundle file, keyspace, and table name you want to create.

# Setup

In [ ]:
!pip3 install cassandra-driver matplotlib sentence-transformers

# Imports

In [ ]:
import os

from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

from PIL import Image
from sentence_transformers import SentenceTransformer
from matplotlib import pyplot as plt
from matplotlib import image as mpimg

# Environment Variables

In [ ]:
SECURE_CONNECT_BUNDLE_PATH = 'secure-connect-vector-search-db.zip'
ASTRA_CLIENT_ID = 'XXX'
ASTRA_CLIENT_SECRET = 'XXX'
KEYSPACE_NAME = 'vsearch'
TABLE_NAME = 'images'

# Connect to Astra DB

In [ ]:
cloud_config = {
   'secure_connect_bundle': SECURE_CONNECT_BUNDLE_PATH
}
auth_provider = PlainTextAuthProvider(ASTRA_CLIENT_ID, ASTRA_CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

# Drop / Create Schema

In [ ]:
print(f"Creating table {TABLE_NAME} in keyspace {KEYSPACE_NAME}")
session.execute(f"CREATE TABLE IF NOT EXISTS {KEYSPACE_NAME}.{TABLE_NAME} (id int PRIMARY KEY, name TEXT, description TEXT, item_vector VECTOR<FLOAT, 512>)")

print(f"Creating index image_ann_index on table {TABLE_NAME} and inserting example data")
session.execute(f"CREATE CUSTOM INDEX IF NOT EXISTS image_ann_index ON {KEYSPACE_NAME}.{TABLE_NAME}(item_vector) USING 'StorageAttachedIndex'")

print(f"Truncate table {TABLE_NAME} in keyspace {KEYSPACE_NAME}")
session.execute(f"TRUNCATE TABLE {KEYSPACE_NAME}.{TABLE_NAME}")


# Load CLIP model

In [ ]:
model = SentenceTransformer('clip-ViT-B-32')

# Generate embeddings from images and load the table with data

In [ ]:
img_emb1 = model.encode(Image.open('one.jpg'))
img_emb2 = model.encode(Image.open('two.jpg'))
img_emb3 = model.encode(Image.open('three.jpg'))
img_emb4 = model.encode(Image.open('four.jpg'))

image_data = [
    (1, 'one.jpg', 'description1', img_emb1.tolist()),
    (2, 'two.jpg', 'description2', img_emb2.tolist()),
    (3, 'three.jpg', 'description3', img_emb3.tolist()),
    (4, 'four.jpg', 'description4', img_emb4.tolist())
]

for image in image_data:
    session.execute(f"INSERT INTO {KEYSPACE_NAME}.{TABLE_NAME} (id, name, description, item_vector) VALUES {image}")

# Generate embeddings from query string

In [ ]:
query_string = "a house with a swimming pool"
text_emb = model.encode(query_string)
print(f"model provided embeddings for the string: 'a house with a swimming pool': {text_emb.tolist()}")


# Vector search the image that shows a house with a swimming pool

In [ ]:
print(f"Astra DB vector search query: SELECT name, description, item_vector FROM {KEYSPACE_NAME}.{TABLE_NAME} ORDER BY item_vector ANN OF {text_emb.tolist()} LIMIT 1")
for row in session.execute(f"SELECT name, description, item_vector FROM {KEYSPACE_NAME}.{TABLE_NAME} ORDER BY item_vector ANN OF {text_emb.tolist()} LIMIT 1"):
    print("\t" + str(row))
    plt.title(row.name)
    image = mpimg.imread(row.name)
    plt.imshow(image)
    plt.show()